In [4]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
# Запрос к API

In [6]:
r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?q=sunflowers')
res = r.json()

In [7]:
pd.Series(res['objectIDs']).shape

(85,)

In [8]:
# Получение информаци к нужному объекту

In [9]:
r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/696949')
obj_test = r_test.json()
obj_test

{'message': 'Not a valid object'}

In [10]:
# Получить информацию обо всех объектах и сложить в DataFrame

In [11]:
columns_df = ['objectID', 'title', 'artistDisplayName',
              'department', 'objectBeginDate', 'objectEndDate',
              'period', 'objectName', 'culture']

In [12]:
df = pd.DataFrame(columns=columns_df)
for i in range(len(res['objectIDs'])):
    local_data = pd.DataFrame(columns=columns_df)
    r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/' 
                          + str(res['objectIDs'][i]))
    obj_test = r_test.json()
    try:
        for j in range(len(columns_df)):
            local_data[columns_df[j]] = pd.Series(obj_test[columns_df[j]]).values
    except:
        continue
    df = df.append(local_data)

In [13]:
df = df.reset_index()

In [14]:
df

,index,objectID,title,artistDisplayName,department,objectBeginDate,objectEndDate,period,objectName,culture
0,0,436524,Sunflowers,Vincent van Gogh,European Paintings,1887,1887,,Painting,
1,0,484935,Sunflowers,Edward McKnight Kauffer,Modern and Contemporary Art,1921,1921,,Painting,
2,0,437112,Bouquet of Sunflowers,Claude Monet,European Paintings,1881,1881,,Painting,
3,0,210191,Vase with sunflowers,Rozenburg Plateelfabriek,European Sculpture and Decorative Arts,1896,1896,,Vase,
4,0,431264,Sunflowers,Lily Converse,Drawings and Prints,1941,1941,,Print,
...,...,...,...,...,...,...,...,...,...,...
77,0,437980,Cypresses,Vincent van Gogh,European Paintings,1889,1889,,Painting,
78,0,724844,Design for Embroidery with Two Large Garlands ...,Italian School,Drawings and Prints,1845,1905,,Drawing Ornament & Architecture,
79,0,834585,"A Life in a Year–The Garden–Noon, from ""Pictur...",Arthur Boyd Houghton,Drawings and Prints,1866,1874,,Print,
80,0,834580,"A Life in a Year–The Garden–Noon, from ""Pictur...",Arthur Boyd Houghton,Drawings and Prints,1866,1874,,Print,


In [15]:
df.query('culture == "Japan"')

,index,objectID,title,artistDisplayName,department,objectBeginDate,objectEndDate,period,objectName,culture
29,0,57922,Incense box,,Asian Art,1700,1799,Edo period (1615–1868),Incense box,Japan


In [16]:
######################################################################################################################

In [17]:
# Выполните поисковый запрос для департамента "Asian art" (departmentId = 6) с тегом "cat" и запишите id объектов в cat_ids. 
# Используйте цикл из шага 4, чтобы получить данные о первой 1000 объектов из списка.

In [18]:
r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?departmentId=6&q=cat')
cats = r.json()
df_cats = pd.DataFrame(columns=columns_df)
for i in range(1000):
    local_data = pd.DataFrame(columns=columns_df)
    r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/' 
                          + str(cats['objectIDs'][i]))
    obj_test = r_test.json()
    try:
        for j in range(len(columns_df)):
            local_data[columns_df[j]] = pd.Series(obj_test[columns_df[j]]).values
    except:
        continue
    df_cats = df_cats = df_cats.append(local_data)

In [19]:
df_cats = df_cats.reset_index()

In [20]:
df_cats = df_cats.drop(columns=['index'])

In [21]:
df_cats.head()

,objectID,title,artistDisplayName,department,objectBeginDate,objectEndDate,period,objectName,culture
0,49698,Cat,Zhang Yuguang,Asian Art,1900,1968,,Folding fan mounted as an album leaf,China
1,49470,Cat,Wang Li,Asian Art,1813,1879,Qing dynasty (1644–1911),Hanging scroll,China
2,53222,Musk Cat,Uto Gyoshi,Asian Art,1550,1599,Muromachi period (1392–1573),Hanging scroll,Japan
3,36221,Cat,Wang Yun,Asian Art,1900,1933,,Hanging scroll,China
4,60873,Burial Crown,,Asian Art,960,1279,Song dynasty (960–1279) or later,Burial crown,China


In [22]:
df_cats.groupby('period', as_index=False).agg({'objectID': 'count'}).sort_values('objectID')

,period,objectID
56,Momoyama (1573–1615) or Edo (1615–1868) period,1
63,Muromachi period (1392–1573) or Momoyama perio...,1
100,Yuan (1271–1368),1
61,Muromachi (1392–1573)–Edo period (1615–1868),1
60,Muromachi (1392–1573),1
...,...,...
62,Muromachi period (1392–1573),30
73,Qing dynasty (1644–1911),49
41,Ming dynasty (1368–1644),49
0,,92
